In [ ]:
# installing pyspark (using pip install pyspark [in the termional])
# installing psycopg2 in terminal
# install java 8
# Posgres jdbc driver(dowlaod and keepin work space directory)

In [2]:
# Import necesary libraries
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameWriter
from pyspark.sql.functions import monotonically_increasing_id
import os
import psycopg2
